## Importing packages

In [1]:
import random
import math
import pulp

In [2]:
N_JOBS = 7
N_MACHINES = 4

## Generating data for job-scheduling

In [3]:
def generateData(seed=0):
    PROC = [ [ 0 for m in range(N_MACHINES) ] for j in range(N_JOBS) ]
    random.seed(seed)
    for j in range(N_JOBS):
        for m in range(N_MACHINES):
            PROC[j][m] = random.randint(1,9)
    return PROC

In [4]:
PROC = generateData(8610)

In [5]:
PROC

[[8, 4, 1, 5],
 [9, 9, 5, 1],
 [1, 6, 6, 6],
 [8, 8, 6, 6],
 [5, 7, 4, 5],
 [9, 9, 1, 8],
 [7, 6, 6, 7]]

In [6]:
def val(x):
    return int(pulp.value(x))

In [7]:
prob = pulp.LpProblem("JobScheduling",pulp.LpMinimize)

`J[j]` defines the sequencing of jobs 'j' and is just a permutation of the job number:

In [8]:
JJ = pulp.LpVariable.dicts("J", (range(N_JOBS), range(N_JOBS)),
                           lowBound=0, upBound=1, cat='Integer')

In [9]:
def job(n):
    for j in range(N_JOBS):
        if val(JJ[n][j])==1:
            return j

In [10]:
def jobSequence():
    acc = []
    for j in range(N_JOBS):
        for jj in range(N_JOBS):
            if pulp.value(JJ[j][jj])==1:
                acc.append(jj)
    return acc

In [11]:
def proc(job, machine):
    global PROC
    return pulp.lpSum( [ PROC[j][machine] * JJ[j][job] for j in range(N_JOBS) ] ) 

`WAIT[j][m]` describes the wait time of job `j` before machine `m`

In [12]:
Wait = pulp.LpVariable.dicts("WAIT", (range(N_JOBS), range(N_MACHINES)),
                          lowBound=0, cat='Integer')

`IDLE[j][m]` describes the idle time of machine `m` before processing job `j` 

In [13]:
Idle = pulp.LpVariable.dicts("IDLE", (range(N_JOBS), range(N_MACHINES)),
                             lowBound=0, cat='Integer')

`START[j][m]` describes the start time of machine `m` processing job `j`

In [14]:
Start = pulp.LpVariable.dicts("START", (range(N_JOBS), range(N_MACHINES)),
                          lowBound=0, cat='Integer')

`STOP[j][m]` describes the stop time of machine `m` after processing job `j`

In [15]:
Stop = pulp.LpVariable.dicts("STOP", (range(N_JOBS), range(N_MACHINES)),
                          lowBound=0, cat='Integer')

In [16]:
prob += Stop[N_JOBS-1][N_MACHINES-1]

In [17]:
for j in range(N_JOBS):
    prob += pulp.lpSum( [ JJ[j][jj] for jj in range(N_JOBS) ] ) == 1
    prob += pulp.lpSum( [ JJ[jj][j] for jj in range(N_JOBS) ] ) == 1

In [18]:
for m in range(N_MACHINES):
    for j in range(N_JOBS):
        prob += pulp.lpSum( [ Idle[ji][m] + proc(ji, m) for ji in range(j) ] ) + Idle[j][m] == Start[j][m]

In [19]:
for m in range(N_MACHINES):
    for j in range(N_JOBS):
        prob += pulp.lpSum( [ Wait[j][mi] + proc(j, mi) for mi in range(m) ] ) + Wait[j][m] == Start[j][m]

In [20]:
for j in range(N_JOBS):
    for m in range(N_MACHINES):
        prob += Start[j][m] + proc(j,m) == Stop[j][m]

In [21]:
solvers = pulp.listSolvers(onlyAvailable=True) 
solver = pulp.getSolver(solvers[0], msg=0)
prob.solve(solver)

1

In [22]:
def schedule():
    row = ""
    row += '|---------|'
    for m in range(N_MACHINES):
        row += f"------------|" \
                f"------------|"
    row += '\n'
    row += '|         |'
    for m in range(N_MACHINES):
        row += f"            |" \
                f" Machine: {m:1d} |"
    row += '\n'
    for j in range(N_JOBS):
        row += '|---------|'
        for m in range(N_MACHINES):
            row += f"------------|" \
                   f"------------|"
        row += '\n'
        row += '|         |'
        for m in range(N_MACHINES):
            row += f"       {' ':4s} | " \
                   f"Idle: {val(Idle[j][m]):4d} |"
        row += '\n'
        row += '|---------|'
        for m in range(N_MACHINES):
            row += f"------------|" \
                   f"------------|"
        row += '\n'
        row += '|         |'
        for m in range(N_MACHINES):
            row += f"       {' ':4s} | " \
                   f"Start:{val(Start[j][m]):4d} |"  
        row += '\n' 
        row += f'| Job: {job(j):2d} |'
        for m in range(N_MACHINES):
            row += f" Wait: {val(Wait[j][m]):4d} | " \
                   f"Proc: {val(proc(j,m)):4d} |"
        row += '\n'
        row += '|         |'
        for m in range(N_MACHINES):
            row += f"       {' ':4s} | " \
                   f"Stop: {val(Stop[j][m]):4d} |"  
        row += '\n' 
    row += '|---------|'
    for m in range(N_MACHINES):
        row += f"------------|" \
                f"------------|"
    row += '\n'
    return row

The processing times per job and machine:

In [23]:
PROC

[[8, 4, 1, 5],
 [9, 9, 5, 1],
 [1, 6, 6, 6],
 [8, 8, 6, 6],
 [5, 7, 4, 5],
 [9, 9, 1, 8],
 [7, 6, 6, 7]]

The optimal job schedule:

In [24]:
print(jobSequence())

[6, 5, 0, 3, 2, 4, 1]


The processing time of the optimal job schedule:

In [25]:
int(pulp.value(prob.objective))

59

The optimal job schedule in detail:

In [26]:
print(schedule())

|---------|------------|------------|------------|------------|------------|------------|------------|------------|
|         |            | Machine: 0 |            | Machine: 1 |            | Machine: 2 |            | Machine: 3 |
|---------|------------|------------|------------|------------|------------|------------|------------|------------|
|         |            | Idle:    0 |            | Idle:    2 |            | Idle:    8 |            | Idle:   14 |
|---------|------------|------------|------------|------------|------------|------------|------------|------------|
|         |            | Start:   0 |            | Start:   2 |            | Start:   8 |            | Start:  14 |
| Job:  6 | Wait:    0 | Proc:    1 | Wait:    1 | Proc:    6 | Wait:    0 | Proc:    6 | Wait:    0 | Proc:    6 |
|         |            | Stop:    1 |            | Stop:    8 |            | Stop:   14 |            | Stop:   20 |
|---------|------------|------------|------------|------------|---------

## Greedy Algorithm

In [27]:
def val(x):
    return int(x)

def greedy_algorithm(PROC):
    N_JOBS = len(PROC)
    N_MACHINES = len(PROC[0])
    
    
    Idle = [[0 for _ in range(N_MACHINES)] for _ in range(N_JOBS)]
    Start = [[0 for _ in range(N_MACHINES)] for _ in range(N_JOBS)]
    Stop = [[0 for _ in range(N_MACHINES)] for _ in range(N_JOBS)]
    
    
    machine_times = [0] * N_MACHINES
    
    
    for j in range(N_JOBS):
        for m in range(N_MACHINES):
            if m == 0:
                Start[j][m] = machine_times[m]
            else:
                Start[j][m] = max(machine_times[m], Stop[j][m-1])
            
            Idle[j][m] = Start[j][m] - (machine_times[m] if m == 0 else Stop[j][m-1])
            Stop[j][m] = Start[j][m] + PROC[j][m]
            machine_times[m] = Stop[j][m]
    
    return Idle, Start, Stop, PROC

In [28]:
def schedule(N_JOBS, N_MACHINES, Idle, Start, Stop, PROC):
    row = ""
    row += '|---------|'
    for m in range(N_MACHINES):
        row += f"------------|" \
                f"------------|"
    row += '\n'
    row += '|         |'
    for m in range(N_MACHINES):
        row += f"            |" \
                f" Machine: {m:1d} |"
    row += '\n'
    for j in range(N_JOBS):
        row += '|---------|'
        for m in range(N_MACHINES):
            row += f"------------|" \
                   f"------------|"
        row += '\n'
        row += '|         |'
        for m in range(N_MACHINES):
            row += f"       {' ':4s} | " \
                   f"Idle: {val(Idle[j][m]):4d} |"
        row += '\n'
        row += '|---------|'
        for m in range(N_MACHINES):
            row += f"------------|" \
                   f"------------|"
        row += '\n'
        row += '|         |'
        for m in range(N_MACHINES):
            row += f"       {' ':4s} | " \
                   f"Start:{val(Start[j][m]):4d} |"  
        row += '\n' 
        row += f'| Job: {j:2d} |'
        for m in range(N_MACHINES):
            row += f" Wait: {val(Idle[j][m]):4d} | " \
                   f"Proc: {PROC[j][m]:4d} |"
        row += '\n'
        row += '|         |'
        for m in range(N_MACHINES):
            row += f"       {' ':4s} | " \
                   f"Stop: {val(Stop[j][m]):4d} |"  
        row += '\n' 
    row += '|---------|'
    for m in range(N_MACHINES):
        row += f"------------|" \
                f"------------|"
    row += '\n'
    return row

Idle, Start, Stop, PROC = greedy_algorithm(PROC)
print("Greedy Algorithm Result:")
print("Job Sequence and Detailed Schedule:")
print(schedule(N_JOBS, N_MACHINES, Idle, Start, Stop, PROC))

Greedy Algorithm Result:
Job Sequence and Detailed Schedule:
|---------|------------|------------|------------|------------|------------|------------|------------|------------|
|         |            | Machine: 0 |            | Machine: 1 |            | Machine: 2 |            | Machine: 3 |
|---------|------------|------------|------------|------------|------------|------------|------------|------------|
|         |            | Idle:    0 |            | Idle:    0 |            | Idle:    0 |            | Idle:    0 |
|---------|------------|------------|------------|------------|------------|------------|------------|------------|
|         |            | Start:   0 |            | Start:   8 |            | Start:  12 |            | Start:  13 |
| Job:  0 | Wait:    0 | Proc:    8 | Wait:    0 | Proc:    4 | Wait:    0 | Proc:    1 | Wait:    0 | Proc:    5 |
|         |            | Stop:    8 |            | Stop:   12 |            | Stop:   13 |            | Stop:   18 |
|---------|

### Calculating Overall Processing Time 

In [29]:
stop_times = [
    [8, 12, 13, 18],
    [17, 26, 31, 32],
    [18, 32, 38, 44],
    [26, 40, 46, 52],
    [31, 47, 51, 57],
    [40, 56, 57, 65],
    [47, 62, 68, 75]
]

def calculate_overall_processing_time(stop_times):
    overall_processing_time = max(max(times) for times in stop_times)
    return overall_processing_time

overall_processing_time = calculate_overall_processing_time(stop_times)

print("Overall Processing Time:", overall_processing_time)

Overall Processing Time: 75


## Simulated Annealing Algorithm (Stochastic Optimisation)

In [30]:
def calculate_makespan(job_order, PROC, N_MACHINES):

    machine_times = [0] * N_MACHINES
    
    for job in job_order:
        
        current_start_time = machine_times[0]
        for machine in range(N_MACHINES):
            if machine == 0:
             
                machine_times[machine] += PROC[job][machine]
            else:
                
                machine_times[machine] = max(machine_times[machine], machine_times[machine - 1]) + PROC[job][machine]
    return machine_times[-1]

def initial_solution(N_JOBS):
    return list(range(N_JOBS))

def get_neighbor(solution):

    a, b = random.sample(range(len(solution)), 2)
    solution[a], solution[b] = solution[b], solution[a]
    return solution

def simulated_annealing(PROC, N_JOBS, N_MACHINES, temp=1000, cooling_rate=0.99, stopping_temp=1):
    current_solution = initial_solution(N_JOBS)
    current_cost = calculate_makespan(current_solution, PROC, N_MACHINES)
    while temp > stopping_temp:
        new_solution = get_neighbor(current_solution[:])
        new_cost = calculate_makespan(new_solution, PROC, N_MACHINES)
        if new_cost < current_cost or random.uniform(0, 1) < math.exp((current_cost - new_cost) / temp):
            current_solution, current_cost = new_solution, new_cost
        temp *= cooling_rate
    return current_solution, current_cost


random.seed(8610)
best_solution, best_cost = simulated_annealing(PROC, N_JOBS, N_MACHINES)
print("Best solution:", best_solution)

Best solution: [4, 2, 6, 0, 5, 3, 1]


In [31]:
def calculate_schedule(job_order, PROC, N_JOBS, N_MACHINES):

    Idle = [[0] * N_MACHINES for _ in range(N_JOBS)]
    Start = [[0] * N_MACHINES for _ in range(N_JOBS)]
    Stop = [[0] * N_MACHINES for _ in range(N_JOBS)]

    
    machine_end_times = [0] * N_MACHINES

 
    for job in job_order:
        for m in range(N_MACHINES):
            if m == 0:
                
                Start[job][m] = machine_end_times[m]
            else:
                
                Start[job][m] = max(Stop[job][m-1], machine_end_times[m])

            
            Stop[job][m] = Start[job][m] + PROC[job][m]

            
            if m == 0:
                Idle[job][m] = 0
            else:
                Idle[job][m] = Start[job][m] - machine_end_times[m]

           
            machine_end_times[m] = Stop[job][m]

    return Idle, Start, Stop

best_solution, best_cost = simulated_annealing(PROC, N_JOBS, N_MACHINES)

Idle, Start, Stop = calculate_schedule(best_solution, PROC, N_JOBS, N_MACHINES)

print("Simulated Annealing Result:")
print("Job Sequence and Detailed Schedule:")
print(schedule(N_JOBS, N_MACHINES, Idle, Start, Stop, PROC))

Simulated Annealing Result:
Job Sequence and Detailed Schedule:
|---------|------------|------------|------------|------------|------------|------------|------------|------------|
|         |            | Machine: 0 |            | Machine: 1 |            | Machine: 2 |            | Machine: 3 |
|---------|------------|------------|------------|------------|------------|------------|------------|------------|
|         |            | Idle:    0 |            | Idle:    0 |            | Idle:    0 |            | Idle:    0 |
|---------|------------|------------|------------|------------|------------|------------|------------|------------|
|         |            | Start:  13 |            | Start:  24 |            | Start:  30 |            | Start:  37 |
| Job:  0 | Wait:    0 | Proc:    8 | Wait:    0 | Proc:    4 | Wait:    0 | Proc:    1 | Wait:    0 | Proc:    5 |
|         |            | Stop:   21 |            | Stop:   28 |            | Stop:   31 |            | Stop:   42 |
|-------

### Calculating Overall Processing Time

In [32]:
def calculate_overall_processing_time(Stop):
    max_stop_time = 0
    for job_stops in Stop:
        job_max = max(job_stops)
        if job_max > max_stop_time:
            max_stop_time = job_max
    return max_stop_time

Stop = [
    [5, 12, 16, 21],
    [6, 18, 24, 30],
    [13, 24, 30, 37],
    [21, 28, 31, 42],
    [29, 37, 43, 49],
    [38, 47, 48, 57],
    [47, 56, 61, 62]
]
overall_processing_time = calculate_overall_processing_time(Stop)
print("Overall Processing Time:", overall_processing_time)

Overall Processing Time: 62
